In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Fake_detector').getOrCreate()

%matplotlib inline

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
import pyspark.sql.functions as F
import pandas as pd

In [3]:
#!hdfs dfs -mkdir /user/$USER/fake_review
#!hdfs dfs -ls /user/$USER/
#!hadoop dfs -put /home/$USER/final/test_no_label.csv     /user/$USER/fake_review
#!hadoop dfs -put /home/$USER/final/train1.csv     /user/$USER/fake_review
#!hadoop dfs -put /home/$USER/final/train2.csv     /user/$USER/fake_review

In [3]:
# import data 
df_no_label = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .option("multiLine", True) \
    .option("ignoreTrailingWhiteSpace", True) \
    .csv("/user/ccy/fake_review/test_no_label.csv",inferSchema=True, header=True)

df_train1 = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .option("multiLine", True) \
    .option("ignoreTrailingWhiteSpace", True) \
    .csv("/user/ccy/fake_review/train1.csv",inferSchema=True, header=True)

df_train2 = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .option("multiLine", True) \
    .option("ignoreTrailingWhiteSpace", True) \
    .csv("/user/ccy/fake_review/train2.csv",inferSchema=True, header=True)

In [4]:
df_no_label.show(10)

+-----+-------+-------+------+-----+-------------------+--------------------+
|ex_id|user_id|prod_id|rating|label|               date|              review|
+-----+-------+-------+------+-----+-------------------+--------------------+
|    6|    929|      0|   4.0| null|2009-08-25 00:00:00|Let me start with...|
|    9|    932|      0|   5.0| null|2014-05-09 00:00:00|Stopped in for lu...|
|   14|    937|      0|   4.0| null|2014-10-15 00:00:00|Tiny little place...|
|   22|    945|      0|   5.0| null|2014-04-10 00:00:00|Food was deliciou...|
|   23|    946|      0|   5.0| null|2014-03-29 00:00:00|Awesome hole in t...|
|   24|    947|      0|   5.0| null|2014-03-21 00:00:00|I love this place...|
|   25|    948|      0|   5.0| null|2014-03-08 00:00:00|The food is amazi...|
|   31|    954|      0|   4.0| null|2013-10-02 00:00:00|Novelty meets Med...|
|   33|    956|      0|   5.0| null|2013-08-22 00:00:00|If you're looking...|
|   34|    957|      0|   4.0| null|2013-08-02 00:00:00|Great fi

In [5]:
df_train1.show(10)

+-----+-------+-------+------+-----+-------------------+--------------------+
|ex_id|user_id|prod_id|rating|label|               date|              review|
+-----+-------+-------+------+-----+-------------------+--------------------+
|    0|    923|      0|   3.0|    1|2014-12-08 00:00:00|The food at snack...|
|    1|    924|      0|   3.0|    1|2013-05-16 00:00:00|This little place...|
|    2|    925|      0|   4.0|    1|2013-07-01 00:00:00|ordered lunch for...|
|    3|    926|      0|   4.0|    1|2011-07-28 00:00:00|This is a beautif...|
|    4|    927|      0|   4.0|    1|2010-11-01 00:00:00|Snack is great pl...|
|    5|    928|      0|   4.0|    1|2009-09-02 00:00:00|A solid 4 stars f...|
|    7|    930|      0|   4.0|    1|2007-05-20 00:00:00|Love this place! ...|
|    8|    931|      0|   4.0|    1|2005-12-27 00:00:00|My friend and I w...|
|   10|    933|      0|   5.0|    1|2014-01-21 00:00:00|pretty cool place...|
|   12|    935|      0|   5.0|    1|2011-01-31 00:00:00|Fabulous

In [6]:
df_train2.show(10)

+-----+-------+-------+------+-----+-------------------+--------------------+
|ex_id|user_id|prod_id|rating|label|               date|              review|
+-----+-------+-------+------+-----+-------------------+--------------------+
|   11|    934|      0|   5.0|    1|2014-01-20 00:00:00|all around good p...|
|   17|    940|      0|   4.0|    0|2014-09-16 00:00:00|For lunch, my fri...|
|   20|    943|      0|   5.0|    0|2014-05-24 00:00:00|Some good Big Gre...|
|   30|    953|      0|   4.0|    0|2013-10-17 00:00:00|So... as you may ...|
|   43|    966|      0|   3.0|    0|2012-12-19 00:00:00|I don't understan...|
|   53|    976|      0|   5.0|    0|2012-05-31 00:00:00|YUMS! I just trie...|
|   56|    979|      0|   4.0|    0|2012-04-11 00:00:00|Delicious. ��Get ...|
|   64|    987|      0|   4.0|    0|2011-12-18 00:00:00|We would love to ...|
|   65|    988|      0|   3.0|    0|2011-12-13 00:00:00|Snack is an adora...|
|   73|    996|      0|   4.0|    0|2011-09-18 00:00:00|Explorin

In [7]:
print("no_label:", df_no_label.count())
print("train1:", df_train1.count())
print("train2:", df_train2.count())

no_label: 72165
train1: 250874
train2: 35918


In [7]:
df_train1.select("label").groupby("label").count().show()

+-----+------+
|label| count|
+-----+------+
|    1| 25819|
|    0|225055|
+-----+------+



In [4]:
df_train = df_train1.drop(*("ex_id","user_id","prod_id","rating","date"))
df_test  = df_train2.drop(*("ex_id","user_id","prod_id","rating","date"))

In [5]:
def wordopt(df):
  #  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    df = df.withColumn("review", regexp_replace(col("review"), '\[.*?\]', " "))     
    df = df.withColumn("review", regexp_replace(col("review"), "\\W", " "))   
    df = df.withColumn("review", regexp_replace(col("review"), 'https?://\S+|www\.\S+', " "))   
    df = df.withColumn("review", regexp_replace(col("review"), '<.*?>+', " "))       
    df = df.withColumn("review", regexp_replace(col("review"), '\n', " "))    
    df = df.withColumn("review", regexp_replace(col("review"), '\w*\d\w*', " "))
    df = df.select("label", lower(col("review")).alias("review"))
    df = df.withColumn("review", regexp_replace(col("review"), "[^A-Za-z]", " "))
    return df

df_train = wordopt(df_train)
df_test = wordopt(df_test)

In [10]:
#df_train = df_train.withColumn("review", regexp_replace(col("review"), "[^A-Za-z]", " "))
#df_test = df_test.withColumn("review", regexp_replace(col("review"), "[^A-Za-z]", " "))

In [6]:
df_train.show(3)

+-----+--------------------+
|label|              review|
+-----+--------------------+
|    1|the food at snack...|
|    1|this little place...|
|    1|ordered lunch for...|
+-----+--------------------+
only showing top 3 rows



## Build Pipeline

In [7]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.feature import HashingTF, IDF

tokenizer = Tokenizer(inputCol="review", outputCol="words")  #tokenize words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")  #remove stop words
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")

word2Vec = Word2Vec(vectorSize=200, inputCol="filtered", outputCol="features")

In [8]:
# train 
pipeline = Pipeline(stages=[tokenizer, remover,hashingTF, idf])
train = pipeline.fit(df_train).transform(df_train)
train = train.drop(*("review", "words"))

In [9]:
# test
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf])
test = pipeline.fit(df_test).transform(df_test)
test = test.drop(*("review", "words"))

In [10]:
train.select("label").groupby("label").count().show()

+-----+------+
|label| count|
+-----+------+
|    1| 25819|
|    0|225055|
+-----+------+



In [11]:
test.select("label").groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 3648|
|    0|32270|
+-----+-----+



### Oversampling 

In [12]:
major_df = train.filter(col("label") == 0)
minor_df = train.filter(col("label") == 1)
ratio = int(major_df.count()/minor_df.count())

In [13]:
a = range(ratio)
# duplicate the minority rows
oversampled_df = minor_df.withColumn("dummy", explode(array([lit(x) for x in a]))).drop('dummy')
# combine both oversampled minority rows and previous majority rows 
combined_df = major_df.unionAll(oversampled_df)
combined_df.show(5)

+-----+--------------------+--------------------+--------------------+
|label|            filtered|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+
|    0|[braised, lamb, s...|(262144,[19889,22...|(262144,[19889,22...|
|    0|[spot, close, job...|(262144,[2437,863...|(262144,[2437,863...|
|    0|[needed, quick, ,...|(262144,[11996,16...|(262144,[11996,16...|
|    0|[artichoke, chick...|(262144,[84488,11...|(262144,[84488,11...|
|    0|[needed, quick, b...|(262144,[14,11996...|(262144,[14,11996...|
+-----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [14]:
combined_df = combined_df.withColumn("label",combined_df.label.cast('float'))

In [15]:
combined_df.select("label").groupby("label").count().show()

+-----+------+
|label| count|
+-----+------+
|  1.0|206552|
|  0.0|225055|
+-----+------+



## Undersampling

In [16]:
sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df_2 = sampled_majority_df.unionAll(minor_df)

In [17]:
combined_df_2.select("label").groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|25819|
|    0|28234|
+-----+-----+



In [18]:
combined_df_2.show(10)

+-----+--------------------+--------------------+--------------------+
|label|            filtered|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+
|    0|[delicious, lamb,...|(262144,[117396,1...|(262144,[117396,1...|
|    0|[tiny, cafe, thom...|(262144,[5987,863...|(262144,[5987,863...|
|    0|[impressed, , fir...|(262144,[5795,912...|(262144,[5795,912...|
|    0|[lacklustre, serv...|(262144,[4200,139...|(262144,[4200,139...|
|    0|[greek, , huge, f...|(262144,[19823,42...|(262144,[19823,42...|
|    0|[followed, great,...|(262144,[5381,104...|(262144,[5381,104...|
|    0|[trying, wait, he...|(262144,[31463,36...|(262144,[31463,36...|
|    0|[stumbled, upon, ...|(262144,[6981,858...|(262144,[6981,858...|
|    0|[friday, night, ,...|(262144,[14,991,8...|(262144,[14,991,8...|
|    0|[small, simple, g...|(262144,[19823,34...|(262144,[19823,34...|
+-----+--------------------+--------------------+--------------------+
only s

## Model

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics

In [20]:
#Logistic Regression Model
lr1 = LogisticRegression() 

# Predict each point's label and show the results.
lrm1 = lr1.fit(combined_df)
predictions = lrm1.transform(test)

In [21]:
#print evaluation metrics
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

0.721615902889916
0.7701399610539134


In [22]:
evaluator_bi = BinaryClassificationEvaluator(metricName='areaUnderROC')
auc = evaluator_bi.evaluate(predictions)
auc

0.631301392717162

In [23]:
predictions.select("prediction").groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|26277|
|       1.0| 9641|
+----------+-----+



## Add Class Weight

In [24]:
balancingRatio = train.filter(col("label") == 0).count() / train.count()

calculateWeights = udf(lambda x: 1 * balancingRatio if x == 0 else (1 * (1.0 - balancingRatio)))
weightedDataset = train.withColumn("classWeightCol", calculateWeights("label"))
weightedDataset = weightedDataset.withColumn("classWeightCol",weightedDataset.classWeightCol.cast('double'))

In [52]:
weightedDataset.show(2)

+-----+--------------------+--------------------+--------------------+-------------------+
|label|            filtered|         rawFeatures|            features|     classWeightCol|
+-----+--------------------+--------------------+--------------------+-------------------+
|    1|[food, snack, sel...|(262144,[2437,978...|(262144,[2437,978...|0.10291620494750353|
|    1|[little, place, s...|(262144,[19862,24...|(262144,[19862,24...|0.10291620494750353|
+-----+--------------------+--------------------+--------------------+-------------------+
only showing top 2 rows



In [105]:
#Logistic Regression Model
#lr = LogisticRegression().setWeightCol("classWeightCol")

lr = LogisticRegression(weightCol="classWeightCol")

# Predict each point's label and show the results.
lrm = lr.fit(weightedDataset)
predictions = lrm.transform(test)

In [106]:
predictions.select("prediction").groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|34482|
|       1.0| 1436|
+----------+-----+



In [107]:
evaluator_bi = BinaryClassificationEvaluator(metricName='areaUnderROC')
auc = evaluator_bi.evaluate(predictions)
auc

0.6375750588510144

In [56]:
#print evaluation metrics
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

0.8703157191380366
0.8442522330060832


## Random Forest 

In [53]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from sklearn.model_selection import RandomizedSearchCV

In [54]:
# Train model with Training Data
rf = RandomForestClassifier()
rfModel = rf.fit(combined_df_2)
predictions = rfModel.transform(test)

In [55]:
predictions.select("prediction").groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|35891|
|       1.0|   27|
+----------+-----+



In [56]:
evaluator_bi = BinaryClassificationEvaluator(metricName='areaUnderROC')
auc = evaluator_bi.evaluate(predictions)
auc

0.6413191159841309

In [57]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

0.8979063422239546
0.8503214121875211


## SVM

In [25]:
from pyspark.ml.classification import LinearSVC

In [71]:
lsvc = LinearSVC()
rfcm = lsvc.fit(combined_df_2)
predictions = rfcm.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

In [ ]:
predictions.select("prediction").groupby("prediction").count().show()

## Test

In [104]:
df_no_label.show(5)

+-----+-------+-------+------+-----+-------------------+--------------------+
|ex_id|user_id|prod_id|rating|label|               date|              review|
+-----+-------+-------+------+-----+-------------------+--------------------+
|    6|    929|      0|   4.0| null|2009-08-25 00:00:00|Let me start with...|
|    9|    932|      0|   5.0| null|2014-05-09 00:00:00|Stopped in for lu...|
|   14|    937|      0|   4.0| null|2014-10-15 00:00:00|Tiny little place...|
|   22|    945|      0|   5.0| null|2014-04-10 00:00:00|Food was deliciou...|
|   23|    946|      0|   5.0| null|2014-03-29 00:00:00|Awesome hole in t...|
+-----+-------+-------+------+-----+-------------------+--------------------+
only showing top 5 rows



In [105]:
table = df_no_label.select("review")

In [27]:
def wordopt(df):
  #  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    df = df.withColumn("review", regexp_replace(col("review"), '\[.*?\]', " "))     
    df = df.withColumn("review", regexp_replace(col("review"), "\\W", " "))   
    df = df.withColumn("review", regexp_replace(col("review"), 'https?://\S+|www\.\S+', " "))   
    df = df.withColumn("review", regexp_replace(col("review"), '<.*?>+', " "))       
    df = df.withColumn("review", regexp_replace(col("review"), '\n', " "))    
    df = df.withColumn("review", regexp_replace(col("review"), '\w*\d\w*', " "))
    df = df.select(lower(col("review")).alias("review"))
    df = df.withColumn("review", regexp_replace(col("review"), "[^A-Za-z]", " "))
    
    return df

In [108]:
table = wordopt(table)
table.show(5)

+--------------------+
|              review|
+--------------------+
|let me start with...|
|stopped in for lu...|
|tiny little place...|
|food was deliciou...|
|awesome hole in t...|
+--------------------+
only showing top 5 rows



In [109]:
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf])
table2 = pipeline.fit(table).transform(table)
table2 = table2.drop(*("review", "words"))

In [110]:
table2.show(5)

+--------------------+--------------------+--------------------+
|            filtered|         rawFeatures|            features|
+--------------------+--------------------+--------------------+
|[let, start, shou...|(262144,[9886,125...|(262144,[9886,125...|
|[stopped, lunch, ...|(262144,[24113,27...|(262144,[24113,27...|
|[tiny, little, pl...|(262144,[54425,61...|(262144,[54425,61...|
|[food, delicious,...|(262144,[22323,24...|(262144,[22323,24...|
|[awesome, hole, w...|(262144,[14,7336,...|(262144,[14,7336,...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [112]:
predictions = lrm.transform(table2)

In [114]:
predictions.select("prediction").groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|69235|
|       1.0| 2930|
+----------+-----+



## Reveiws from Meet SPA and Tanta

In [47]:
review = spark.createDataFrame(
    [
        ("Meet SPA", "This place is brand new, and it is absolutely gorgeous inside. They are still building their website, so if you have any question, just call ahead.  I received a massage from Nina, it took me to a different planet. I fell asleep twice which is definitely a compliment. Pricing is very fair, and they have an array of services to choose from."),
        ("Meet SPA","This spa is absolutely gorgeous and the owner and staff are incredibly kind and gracious. This was one of--if not the best--massage snd spa treatment I have ever experienced. I promise--you will not be disappointed!!!!"),
        ("Meet SPA","I found this place on Yelp. I was looking for a same day couples massage in the West Loop. I loved that the place was super clean and very easy to book. I tried the hot stone massage. They also play great relaxing music. I recommend calling to book an appointment. I couldn't figure out the website, however I received great customer service over the phone."),
        ("Tanta","Came to Tanta for dinner with a friend  on the weekend . Very warm service and see up sitting at the bar where the bartenders were wonderful. Had some fancy cocktails , Ceviche and the Peruvian steak . Everything was tasty and the mood was festive ! Great choice for Latin inspired fusion with some sushi ( Japanese ) inspirations !"),
        ("Tanta", "Try Lomo Saltado, Skirt Steak from the Nikkei bar and don't forget tres leches con lucuma. Maybe one if the best desserts I have tried! I didn't like the music that much but the food still deserves 5 stars"),
        ("Tanta","Loooove this place and definitely coming back again. Came here for a birthday dinner and the service was so nice. They even brought me out a little dessert and candle.The octopus was Devine. Perfectly grilled and cooked. The pork belly was amazing. The chicken skewers were good too but probably not the best dish in comparison to everything else we ordered.")
    ],
    ["id", "review"]
)

In [48]:
review.show()

+--------+--------------------+
|      id|              review|
+--------+--------------------+
|Meet SPA|This place is bra...|
|Meet SPA|This spa is absol...|
|Meet SPA|I found this plac...|
|   Tanta|Came to Tanta for...|
|   Tanta|Try Lomo Saltado,...|
|   Tanta|Loooove this plac...|
+--------+--------------------+



In [52]:
review = wordopt(review) 
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf])
table2 = pipeline.fit(review).transform(review)
table2 = table2.drop(*("review", "words"))

result = lrm1.transform(table2)
result.select("filtered","probability","prediction").show()

+--------------------+--------------------+----------+
|            filtered|         probability|prediction|
+--------------------+--------------------+----------+
|[place, brand, ne...|[0.58334630905852...|       0.0|
|[spa, absolutely,...|[0.49604392569652...|       1.0|
|[found, place, ye...|[0.44975949597738...|       1.0|
|[came, tanta, din...|[0.79013395692662...|       0.0|
|[try, lomo, salta...|[0.64433429476311...|       0.0|
|[loooove, place, ...|[0.70299984302838...|       0.0|
+--------------------+--------------------+----------+

